In [ ]:
# Import libraries

import matplotlib.pyplot as plt
import matplotlib as mpl

import cartopy.crs as ccrs
#from cartopy.util import add_cyclic_point

import xarray as xr

import numpy as np

from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter

from matplotlib import colorbar, colors

from eofs.xarray import Eof


from bozza.function import searching_model_path, SearchingModelPath

In [ ]:
SearchingModelPath.searching_model_path('ts_Amon_MIROC6_historical_r1i1p1f1_gn_185001-201412.nc')

In [ ]:
path_model_ts_amon_mri =  searching_model_path('ts_Amon_MRI-AGCM3-2-S_highresSST-present_r1i1p1f1_gn_201001-201412(3).nc')
path_model_ts_amon_mri_2h = searching_model_path('ts_Amon_MRI-AGCM3-2-H_highresSST-present_r1i1p1f1_gn_195001-201412.nc')

In [ ]:
path_model_ts_amon_hiram_1 = searching_model_path('ts_Amon_HiRAM-SIT-HR_highresSST-present_r1i1p1f1_gn_200801-200812.nc')
path_model_ts_amon_hiram_2 = searching_model_path('ts_Amon_HiRAM-SIT-HR_highresSST-present_r1i1p1f1_gn_200901-200912.nc')
path_model_ts_amon_hiram_3 = searching_model_path('ts_Amon_HiRAM-SIT-HR_highresSST-present_r1i1p1f1_gn_201001-201012.nc')
path_model_ts_amon_hiram_4 = searching_model_path('ts_Amon_HiRAM-SIT-HR_highresSST-present_r1i1p1f1_gn_201101-201112.nc')
path_model_ts_amon_hiram_5 = searching_model_path('ts_Amon_HiRAM-SIT-HR_highresSST-present_r1i1p1f1_gn_201201-201212.nc')
path_model_ts_amon_hiram_6 = searching_model_path('ts_Amon_HiRAM-SIT-HR_highresSST-present_r1i1p1f1_gn_201301-201312.nc')
path_model_ts_amon_hiram_7 = searching_model_path('ts_Amon_HiRAM-SIT-HR_highresSST-present_r1i1p1f1_gn_201401-201412.nc')

In [ ]:
model_hiram_1 = xr.open_dataset(path_model_ts_amon_hiram_1)

model_hiram = xr.concat([
    xr.open_dataset(path_model_ts_amon_hiram_1),
    xr.open_dataset(path_model_ts_amon_hiram_2),
    xr.open_dataset(path_model_ts_amon_hiram_3),
    xr.open_dataset(path_model_ts_amon_hiram_4),
    xr.open_dataset(path_model_ts_amon_hiram_5),
    xr.open_dataset(path_model_ts_amon_hiram_6),
    xr.open_dataset(path_model_ts_amon_hiram_7),
], dim = "time")
model_hiram = model_hiram.sel(time=slice('2010-01','2014-12'))

In [ ]:
temperature_surface_model_hiram = model_hiram.ts - 273.15
temperature_surface_model_hiram

In [ ]:
temperature_surface_model_hiram

In [ ]:
model_mri = xr.open_dataset(path_model_ts_amon_mri)
temperature_surface_mri = model_mri.ts - 273.15
temperature_surface_mri

In [ ]:
print(f'il numero di lon è {len(temperature_surface_mri.lon)} e il numero di lat è {len(temperature_surface_mri.lat)}')

In [ ]:
print(f'il numero di lon è {len(temperature_surface_model_hiram.lon)} e il numero di lat è {len(temperature_surface_model_hiram.lat)}')

In [ ]:
#d1.coords['lon'] = (d1.coords['lon'] + 180) % 360 - 180
#d1 = d1.sortby(d1.lon)

In [ ]:
#fig = plt.figure(figsize=(20, 10))
#p0 = temperature_surface_model_hiram.mean('time').plot(transform=ccrs.PlateCarree(),subplot_kws={'projection': ccrs.Robinson()},
#                            cmap='jet')
#p0.axes.coastlines()
#p0.axes.gridlines()
#plt.rcParams["figure.figsize"] = (10,10)
#plt.title('Temperature' )

In [ ]:
fig = plt.figure(figsize=(20, 10))
p0 = temperature_surface_mri.mean('time').plot(transform=ccrs.PlateCarree(),subplot_kws={'projection': ccrs.Robinson()},
                            cmap='jet')
p0.axes.coastlines()
p0.axes.gridlines()
plt.rcParams["figure.figsize"] = (10,10)
plt.title('Temperature' )

# Project

- The idea is using two or more model try to apply some sort of clustering methods, so i want to divide my project in two part, one is for the spatial clustering, so i want to slice my maps into smaller part and try to cluster it, maybe i can take only one specific periods(2000-2014) take the anomalies.
- and the second part is using time series, so i have to aggregate in the spatial way my info, try to do some sort of time series clustering, something like my data science lab project.

so the first part of the project is trying to subdivide my area in subgrups. the only things that is have to take in mind is that i have to try to take the spacial information with all the data

In [ ]:
model_hiram

In [ ]:
model_mri_interp = model_mri.copy()
model_mri_interp = model_mri_interp.interp(lat=model_hiram.lat, lon=model_hiram.lon, method='linear')

In [ ]:
print(f'il numero di lon è {len(model_mri_interp.lon)} e il numero di lat è {len(model_mri_interp.lat)}')
print(f'il numero di lon è {len(model_hiram.lon)} e il numero di lat è {len(model_hiram.lat)}')

In [ ]:
#model_mri_interp_celsius_mean_time = model_mri_interp.mean('time', keep_attrs=True)
#model_hiram_celsius_mean_time = model_hiram.mean('time', keep_attrs=True)

#ens = model_mri_interp_celsius_mean_time.merge(model_hiram_celsius_mean_time.ts.rename('ts2'))
#ens = ens.drop_vars(('lat_bnds','lon_bnds'))
#ens

In [ ]:
ens = model_mri_interp.merge(model_hiram.ts.rename('ts2'))
ens = ens.drop_vars(('lat_bnds','lon_bnds','time_bnds'))
ens

In [ ]:
len(ens.ts.mean(('time','lon')))

In [ ]:
(ens.ts.mean(('time','lon'))-273).plot()

In [ ]:
ens['ts_anomaly'] = ens.ts - ens.ts.mean(('time','lon'))
ens['ts2_anomaly'] = ens.ts2 - ens.ts2.mean(('time','lon'))
ens = ens.drop_vars(('ts','ts2'))
ens

In [ ]:
ens_mean = ens.to_array(dim='new').mean('new')
ens_stdev = ens.to_array(dim='new2').std('new2')

ens['ens_mean'] = ens_mean  # convert to Celsius
ens['ens_stdev'] = ens_stdev
ens = ens.drop_vars(('ts_anomaly','ts2_anomaly'))

ens

In [ ]:
fig = plt.figure(figsize=(20, 10))
p0 = ens.ens_mean.mean('time').plot(transform=ccrs.PlateCarree(),subplot_kws={'projection': ccrs.Robinson()},
                            cmap='jet')
p0.axes.coastlines()
p0.axes.gridlines()
plt.rcParams["figure.figsize"] = (10,10)
plt.title('Temperature' )